In [1]:
from pathlib import Path
from src.config import get_settings

API_BASE_URL = "http://localhost:8000"
AIRFLOW_URL = "http://localhost:8080"

# Host & port các service
POSTGRES_HOST = "localhost"
POSTGRES_PORT = 5432          # hoặc 5433 nếu muốn connect airflow-db
REDIS_HOST = "localhost"
REDIS_PORT = 6379
OPENSEARCH_HOST = "http://localhost:9200"
OLLAMA_HOST = "http://localhost:11434"

# Cấu hình RAG
TOP_K = 5
USE_HYBRID = True
TIMEOUT = 60

settings = get_settings()
pdf_dir = Path(settings.data.pdf_dir)

pdf_files = list(pdf_dir.glob("*.pdf"))
pdf_paths = [str(f) for f in pdf_files]

print(f"Found {len(pdf_paths)} PDF files:")
pdf_paths

Found 58 PDF files:


['data\\pdf\\Public_427.pdf',
 'data\\pdf\\Public_428.pdf',
 'data\\pdf\\Public_441.pdf',
 'data\\pdf\\Public_442.pdf',
 'data\\pdf\\Public_444.pdf',
 'data\\pdf\\Public_445.pdf',
 'data\\pdf\\Public_446.pdf',
 'data\\pdf\\Public_447.pdf',
 'data\\pdf\\Public_448.pdf',
 'data\\pdf\\Public_449.pdf',
 'data\\pdf\\Public_465.pdf',
 'data\\pdf\\Public_466.pdf',
 'data\\pdf\\Public_468.pdf',
 'data\\pdf\\Public_469.pdf',
 'data\\pdf\\Public_470.pdf',
 'data\\pdf\\Public_471.pdf',
 'data\\pdf\\Public_472.pdf',
 'data\\pdf\\Public_473.pdf',
 'data\\pdf\\Public_474.pdf',
 'data\\pdf\\Public_624.pdf',
 'data\\pdf\\Public_626.pdf',
 'data\\pdf\\Public_634.pdf',
 'data\\pdf\\Public_635.pdf',
 'data\\pdf\\Public_636.pdf',
 'data\\pdf\\Public_637.pdf',
 'data\\pdf\\Public_638.pdf',
 'data\\pdf\\Public_639.pdf',
 'data\\pdf\\Public_640.pdf',
 'data\\pdf\\Public_641.pdf',
 'data\\pdf\\Public_642.pdf',
 'data\\pdf\\Public_643.pdf',
 'data\\pdf\\Public_644.pdf',
 'data\\pdf\\Public_645.pdf',
 'data\\pd

In [2]:
import requests
import pandas as pd
import time
import json
from pathlib import Path
from typing import Dict, Any, List
from datetime import datetime
import socket
from urllib.parse import urlparse

# Đường dẫn files
DATA_DIR = Path("data")
PDF_DIR = DATA_DIR / "pdf"
QUESTION_FILE = DATA_DIR / "question.csv"
OUTPUT_FILE = DATA_DIR / "answers_output.csv"

In [ ]:
def print_header(text: str):
    print("\n" + "=" * 80)
    print(f"  {text}")
    print("=" * 80)

def print_step(step_num: int, text: str):
    print(f"\n{'─' * 80}")
    print(f"📌 BƯỚC {step_num}: {text}")
    print(f"{'─' * 80}")

def print_step(step_num: int, text: str):
    print(f"\n{'─' * 80}")
    print(f"📌 BƯỚC {step_num}: {text}")
    print(f"{'─' * 80}")


def load_questions() -> pd.DataFrame:
    """Đọc câu hỏi từ CSV file."""
    print_step(4, "Đọc câu hỏi từ CSV")

    if not QUESTION_FILE.exists():
        print(f"\n❌ File không tồn tại: {QUESTION_FILE}")
        print("Vui lòng tạo file question.csv với format:")
        print("  Question,A,B,C,D,source_folder")
        return None

    try:
        df = pd.read_csv(QUESTION_FILE)
        print(f"\n✅ Đọc thành công {len(df)} câu hỏi")

        # Kiểm tra columns
        required_cols = ["Question", "A", "B", "C", "D"]
        missing_cols = [col for col in required_cols if col not in df.columns]

        if missing_cols:
            print(f"❌ Thiếu columns: {missing_cols}")
            return None

        # Hiển thị sample
        print("\n📝 Ví dụ câu hỏi đầu tiên:")
        print(f"   Q: {df.iloc[0]['Question']}...")

        return df

    except Exception as e:
        print(f"❌ Lỗi khi đọc CSV: {e}")
        return None

load_questions()


────────────────────────────────────────────────────────────────────────────────
📌 BƯỚC 4: Đọc câu hỏi từ CSV
────────────────────────────────────────────────────────────────────────────────

✅ Đọc thành công 310 câu hỏi

📝 Ví dụ câu hỏi đầu tiên:
   Q: Mối quan hệ giữa độ nhạy tham chiếu, ACS và đặc tính chặn trong hiệu suất máy thu vô tuyến là gì?...


,Question,A,B,C,D
0,"Mối quan hệ giữa độ nhạy tham chiếu, ACS và đặ...",Ba tham số này cùng đảm bảo khả năng chống nhi...,Ba tham số này đều liên quan đến công suất phát,Ba tham số này thay thế lẫn nhau,Ba tham số này chỉ áp dụng cho UE băng 20 MHz
1,Theo thông số kỹ thuật trong tài liệu Public_2...,"Có, vì nằm trong giới hạn cho phép","Không, vì vượt quá độ dày tối đa 0,19 mm","Có, vì máy in nhiệt không giới hạn độ dày",Không xác định được từ thông tin tài liệu
2,"Đối với nghiệp vụ cố định trong dải 1,5 ÷ 30 M...","< 20 kHz , 50 kHz , < 60 kHz , 150 kHz","< 25 kHz , 60 kHz , < 70 kHz , 180 kHz","< 30 kHz , 75 kHz , < 80 kHz , 200 kHz","< 40 kHz , 100 kHz , < 90 kHz , 220 kHz"
3,Mục đích chính của việc ban hành bộ chỉ tiêu k...,Đào tạo công nhân kỹ thuật tại kho,"Làm cơ sở đầu tư, lắp đặt và kiểm soát chất lư...",Thay thế toàn bộ băng chuyền cũ trong kho,Chuẩn hóa quy trình an toàn cháy nổ trong kho
4,Đầu Ra Của Quy Trình Lựa Chọn DBMS Là Gì?,Hợp Đồng Bảo Trì DBMS,Checklist Đánh Giá Kết Thúc Dự Án,DBMS Được Lựa Chọn Đưa Vào CTKT Và Tài Liệu Gi...,Danh Sách Ứng Viên DBMS Chưa Phê Duyệt
...,...,...,...,...,...
305,"Theo tài liệu Public_369, các nhóm đài chính đ...",Chỉ cố định và quảng bá,"Cố định, di động, hàng không, hàng hải","Cố định, mặt đất, di động, vô tuyến xác định, ...","Cố định, di động, vệ tinh"
306,Thiết bị nào có thể duy trì hoạt động trong kh...,Màn hình hiển thị tương tác,Thiết bị lưu trữ có phân quyền,Hệ thống điều khiển trung tâm,Máy chiếu vật thể
307,"Theo tài liệu Public_374, thiết bị dùng trong ...",25 MHz – 500 MHz,25 MHz – 1 GHz,500 MHz – 2 GHz,"2,4 GHz – 5 GHz"
308,"Theo khuyến nghị ITU-T G.826, chiều dài đường ...",12000 km,18500 km,27500 km,35000 km


In [ ]:
def ask_single_question(
    question: str, options: Dict[str, str], source_folder: str = None
) -> Dict[str, Any]:
    """Trả lời một câu hỏi MCQ."""

    payload = {"question": question, "options": options, "top_k": TOP_K, "use_hybrid": USE_HYBRID}

    if source_folder and pd.notna(source_folder):
        payload["source_folder"] = source_folder

    try:
        response = requests.post(f"{API_BASE_URL}/api/v1/ask", json=payload, timeout=TIMEOUT)
        response.raise_for_status()
        return response.json()

    except requests.exceptions.Timeout:
        print(f"    ⏱️  Timeout (>{TIMEOUT}s)")
        return {"error": "timeout", "predicted_option": None}

    except Exception as e:
        print(f"    ❌ Lỗi: {str(e)[:100]}")
        return {"error": str(e), "predicted_option": None}

def answer_all_questions(df: pd.DataFrame) -> pd.DataFrame:
    """Trả lời tất cả câu hỏi trong dataframe."""
    print_step(5, "Trả lời câu hỏi")

    results = []
    total = len(df)

    print(f"\n🎯 Bắt đầu trả lời {total} câu hỏi...\n")

    start_time = time.time()

    for idx, row in df.iterrows():
        q_num = idx + 1

        # Progress
        progress = (q_num / total) * 100
        print(f"[{q_num}/{total}] ({progress:.1f}%) ", end="")
        print(f"Q: {row['Question'][:60]}...")

        # Chuẩn bị options
        options = {"A": str(row["A"]), "B": str(row["B"]), "C": str(row["C"]), "D": str(row["D"])}

        # Lấy source_folder nếu có
        source_folder = row.get("source_folder", None)

        # Gọi API
        q_start = time.time()
        answer_data = ask_single_question(
            question=row["Question"], options=options, source_folder=source_folder
        )
        q_time = time.time() - q_start

        # Parse kết quả
        predicted = answer_data.get("predicted_option", "N/A")
        confidence = answer_data.get("confidence", "unknown")
        reasoning = answer_data.get("reasoning", "")
        error = answer_data.get("error", None)

        # Hiển thị kết quả
        conf_icon = {"high": "🟢", "medium": "🟡", "low": "🔴"}.get(confidence, "⚪")

        print(f"    → Đáp án: {predicted} {conf_icon} ({q_time:.1f}s)")
        print(f"      Suy luận: {reasoning}")
        # Lưu kết quả
        result = {
            "question_number": q_num,
            "question": row["Question"],
            "option_A": row["A"],
            "option_B": row["B"],
            "option_C": row["C"],
            "option_D": row["D"],
            "source_folder": source_folder if pd.notna(source_folder) else "",
            "predicted_answer": predicted,
            "confidence": confidence,
            "reasoning": reasoning[:500] if reasoning else "",  # Giới hạn độ dài
            "processing_time_seconds": round(q_time, 2),
            "error": error if error else "",
            "timestamp": datetime.now().isoformat(),
        }

        # Thêm timing details nếu có
        if "timing" in answer_data:
            timing = answer_data["timing"]
            result["retrieval_time_ms"] = timing.get("retrieval_ms", 0)
            result["generation_time_ms"] = timing.get("generation_ms", 0)

        results.append(result)

        # Ngắt dòng sau mỗi câu
        if q_num % 5 == 0:
            print()

    total_time = time.time() - start_time

    print("\n" + "─" * 80)
    print(f"✅ Hoàn thành! Tổng thời gian: {total_time:.1f}s")
    print(f"⏱️  Trung bình: {total_time / total:.1f}s/câu")
    print("─" * 80)

    return pd.DataFrame(results)

def save_results(results_df: pd.DataFrame):
    """Lưu kết quả ra CSV file."""
    print_step(6, "Lưu kết quả")

    try:
        results_df.to_csv(OUTPUT_FILE, index=False, encoding="utf-8-sig")

        print(f"\n✅ Đã lưu kết quả vào: {OUTPUT_FILE}")
        print(f"📊 Tổng số câu: {len(results_df)}")

        # Thống kê
        conf_counts = results_df["confidence"].value_counts()
        print("\n📈 Thống kê độ tin cậy:")
        for conf, count in conf_counts.items():
            percentage = (count / len(results_df)) * 100
            print(f"  • {conf}: {count} câu ({percentage:.1f}%)")

        # Kiểm tra errors
        errors = results_df[results_df["error"] != ""]
        if len(errors) > 0:
            print(f"\n⚠️  Có {len(errors)} câu bị lỗi:")
            for idx, row in errors.iterrows():
                print(f"  • Câu {row['question_number']}: {row['error']}")

    except Exception as e:
        print(f"❌ Lỗi khi lưu file: {e}")

def display_summary(results_df: pd.DataFrame):
    """Hiển thị tóm tắt kết quả."""
    print_header("📊 TÓM TẮT KẾT QUẢ")

    total = len(results_df)
    successful = len(results_df[results_df["error"] == ""])

    print(f"\n✅ Tổng số câu: {total}")
    print(f"✅ Trả lời thành công: {successful}")
    print(f"❌ Lỗi: {total - successful}")

    # Top 5 câu có confidence cao
    high_conf = results_df[results_df["confidence"] == "high"].head(5)
    if len(high_conf) > 0:
        print(f"\n🟢 Top {len(high_conf)} câu có độ tin cậy cao:")
        for idx, row in high_conf.iterrows():
            print(
                f"  {row['question_number']}. Đáp án {row['predicted_answer']}: {row['question']}..."
            )

    # Câu có confidence thấp
    low_conf = results_df[results_df["confidence"] == "low"]
    if len(low_conf) > 0:
        print(f"\n🔴 {len(low_conf)} câu có độ tin cậy thấp:")
        for idx, row in low_conf.head(3).iterrows():
            print(f"  {row['question_number']}. {row['question']}...")

    print("\n" + "=" * 80)

In [5]:
def main():
    """Hàm chính để chạy toàn bộ pipeline."""

    print_header("🚀 RAG MCQ SYSTEM - AUTOMATIC ANSWERING")
    print(f"\n📅 Bắt đầu: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    # Step 3: Check indexed documents
    # doc_count = check_indexed_documents()
    # if doc_count == 0:
    #     print("\n❌ Không có documents nào trong hệ thống!")
    #     print("Không thể trả lời câu hỏi. Vui lòng xử lý PDF trước!")
    #     return

    # Step 4: Load questions
    questions_df = load_questions()
    if questions_df is None:
        return

    # Step 5: Answer all questions
    results_df = answer_all_questions(questions_df)

    # Step 6: Save results
    save_results(results_df)

    # Display summary
    display_summary(results_df)

    print_header("✅ HOÀN TẤT!")
    print(f"\n📁 Kết quả đã được lưu tại: {OUTPUT_FILE.absolute()}")
    print(f"📅 Kết thúc: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("\n" + "=" * 80 + "\n")

In [ ]:
if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n\n⚠️  Đã dừng bởi người dùng (Ctrl+C)")
    except Exception as e:
        print(f"\n\n❌ Lỗi không mong đợi: {e}")
        import traceback

        traceback.print_exc()


  🚀 RAG MCQ SYSTEM - AUTOMATIC ANSWERING

📅 Bắt đầu: 2025-11-11 04:41:24

────────────────────────────────────────────────────────────────────────────────
📌 BƯỚC 4: Đọc câu hỏi từ CSV
────────────────────────────────────────────────────────────────────────────────

✅ Đọc thành công 310 câu hỏi

📝 Ví dụ câu hỏi đầu tiên:
   Q: Mối quan hệ giữa độ nhạy tham chiếu, ACS và đặc tính chặn trong hiệu suất máy thu vô tuyến là gì?...

────────────────────────────────────────────────────────────────────────────────
📌 BƯỚC 5: Trả lời câu hỏi
────────────────────────────────────────────────────────────────────────────────

🎯 Bắt đầu trả lời 310 câu hỏi...

[1/310] (0.3%) Q: Mối quan hệ giữa độ nhạy tham chiếu, ACS và đặc tính chặn tr...
    → Đáp án: A 🟢 (12.6s)
[2/310] (0.6%) Q: Theo thông số kỹ thuật trong tài liệu Public_291, máy in nhi...
    → Đáp án: B 🟢 (21.8s)
[3/310] (1.0%) Q: Đối với nghiệp vụ cố định trong dải 1,5 ÷ 30 MHz, khi công s...
    → Đáp án: None 🟡 (32.3s)
[4/310] (1.3%) Q: M